# Projeto 2 - Ciência dos Dados

**Nome**: Jonas da Silva Lopes

**Nome**: William Silva


# Classificador automático de sentimento

## Preparando o ambiente no jupyter:

In [73]:
%%capture
!pip install tweepy
!pip install emoji

In [74]:
import tweepy
import emoji
import math
import os.path
import pandas as pd
import json
import re 
import functools
import operator
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@William48253649***

In [75]:
#Identificador da conta no twitter: @William48253649

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [76]:
#Produto escolhido
produto = 'Nubank'

#Quantidade mínima de mensagens capturadas
n = 500
#Quantidade mínima de mensagens para a base de treinamento
t = 300

#Filtro de língua
lang = 'pt'

Capturando os dados do twitter:

In [77]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower()) #Lower para deixar tudo minúsculo e facilitar a comparação
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [78]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])}).set()
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])}).set()
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Após realizar a classificação manual das mensagens, como irrelevante (0) ou relevante (1), partimos para  a mudança desses valores para algo mais palpável para a análise. Com isso, fazemos a alteração que pode ser vista abaixo.

Mas, para realizar isso, primeiro foi necessário determinar critérios para a classificação:

- Mencionar o produto;
- A menção ao produto deve ser acompanhada de uma opinião;
- A opinião pode ser demonstrada na forma de indagações, reclamações, pode envolver sarcasmo, elogios e sugestões sobre serviços;
- A opinião afirmada deve ser clara;
- Emoctions também representam opiniões.

In [95]:
mensagens = pd.read_excel("Nubank.xlsx")
mensagens.Relevância = mensagens.Relevância.astype('category')
mensagens.Relevância.cat.categories = ('Irrelevante', 'Relevante')

print("A quantidade de cada cada um é: \n\n", mensagens.Relevância.value_counts())

relevante = mensagens[mensagens.Relevância=="Relevante"]
irrelevante = mensagens[mensagens.Relevância=="Irrelevante"]

total = len(relevante) + len(irrelevante)

A quantidade de cada cada um é: 

 Irrelevante    184
Relevante      116
Name: Relevância, dtype: int64


,Treinamento,Relevância
0,rindo mas preocupada pois cada dia parece que ...,Relevante
1,rt @nubank: este é um atendimento real do nuba...,Irrelevante
2,esse nubank é bom mesmo? 🤔,Irrelevante
3,"@brunoln91 @nubank pow @nubank, verdinho e tud...",Irrelevante
4,@kinnybouvier_ @1masterball vo ligar no nubank...,Irrelevante
5,depois dessa vou deixar meu limite negativo no...,Irrelevante
6,eu amo o nubank mesmo https://t.co/rrpb89yes9,Relevante
7,meu pai pediu um cartão novo pra mim pra pegar...,Relevante
8,"quem inventou o nubank nem é gente, é um anjo",Relevante
9,rt @bichabichabcha: @nubank @mabzinhaa antes m...,Irrelevante


In [80]:
''' Função que troca pontuação por espaço '''
def cleanup(text):
    punctuation = '[!\-.:?;/,|@"\'()]'
    pattern = re.compile(punctuation)
    # Abaixo, determina que se troca por espaço
    text_subbed = re.sub(pattern, ' ', text)
    
    emoji_dividir = emoji.get_emoji_regexp().split(text_subbed)
    espaco_dividir = [substr.split() for substr in emoji_dividir]
    split = functools.reduce(operator.concat, espaco_dividir)
    
    return split

# Usando a função apply para fazer a limpeza nas mensagens
nubank_relev = relevante.Treinamento.apply(cleanup)
nubank_irrelev = irrelevante.Treinamento.apply(cleanup)


lista = []
i = 0
for linha in nubank_relev:
    while i < len(linha):
        lista.append(linha[i])
        i += 1
    i = 0
    
lista_nubank_relev = pd.DataFrame(lista) 


##### Frequência Absoluta

In [81]:
lista_nubank_relev[0].value_counts()

nubank            122
o                  58
de                 54
e                  52
eu                 47
que                45
a                  35
meu                34
me                 29
t                  28
co                 28
https              28
um                 27
não                26
é                  25
cartão             23
do                 23
no                 20
uma                16
na                 15
q                  15
pra                15
to                 14
com                14
só                 13
limite             12
mais               12
da                 11
nunca              11
tem                11
                 ... 
chamar              1
resolvi             1
azgxuchgp3          1
inventou            1
carteiro            1
awarenesswoman      1
serem               1
1h                  1
motivo              1
emergências         1
daniele             1
assi                1
gj2uj2obpj          1
boa                 1
agressivas

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [93]:
lista_counts_relevante=(lista_nubank_relev[0].value_counts()).to_dict()
dicionario_counts_relevancia = (mensagens.Relevância.value_counts()).to_dict()



'''
def probabilidade_do_naive(mensagens, lista_counts_relevante, dicionario_counts_relevancia, total):
    mensagens_selecionados
    verossimilhança =
    
    for rel, v in dicionario_counts_relevancia.items():
        if rel == "Relevante"
            total_rel = v
    priori = total_rel/total
    
    
    return verossimilhança*priori
'''

0.38666666666666666

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**